<a href="https://colab.research.google.com/github/assismauro/CursoPython/blob/master/Azimuth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Azimuth app

This notebook solves the following problem:

In an environment, one way to monitor fire is to put high towers in the middle and around the area in a way to track fire events. People stay 24 hours seven days a week at the top of the tower looking for fire flames or smoke. If two people in different towers see the same fire, it's possible to triangulate the azimuths (angle between North and the fire directions) and then to calculate the firing point.

If you provide data about two towers (UTM coordinates) and the azimuth of the fire for each tower, this app calculates the UTM coordinates where the fire is, the distance between the towers and the flames or smoke and plots a diagram showing the results.

One limitation of this "app" has is that the three points coordinates (both towers and file) must be in the same UTM zone.

How to use:

* Click in the Run button (right arrow) below.
* Fill towers data.
* Click Run again.
* Click Calculate! button that appears below the form. 


Enjoy!

In [7]:
import ipywidgets as widgets
from IPython.display import display
from shapely.geometry import Point, LineString
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#@title {run: 'auto'} ## Dados de Entrada
#@markdown ---
 
#@markdown #### A Tower
nomeA = "Ponto Ferruginoso" #@param {type:"string"}
xA = 603484.00 #@param {type:"number"}
yA = 7781714.00 #@param {type:"number"}
azA = 222.00 #@param {type:"number"}
 
#@markdown #### B Tower 
nomeB = "Posto 39" #@param {type:"string"}
xB = 600593.00 #@param {type:"number"}
yB = 7781310.00 #@param {type:"number"}
azB = 170.00 #@param {type:"number"}
 
button = widgets.Button(description="Calculate!")
output = widgets.Output()
 
# trigonometry
def cosdir(azim):
    az = np.radians(azim)
    cosa = np.sin(az)
    cosb = np.cos(az)
    return cosa,cosb
 
def calcIntersection(xA,yA,azA,xB,yB,azB): 
    pointA = Point(xA,yA)
    cosaA, cosbA  = cosdir(azA)
    pointB = Point(xB,yB)
    cosaB, cosbB  = cosdir(azB)
    length = 10000
    endlineA = Point(pointA.x+(length*cosaA), pointA.y+(length*cosbA))
    endlineB = Point(pointB.x+(length*cosaB), pointB.y+(length*cosbB))
    line1 = LineString([pointA,endlineA])
    line2 = LineString([pointB,endlineB])
    intersection = LineString([pointB,endlineB]).intersection(LineString([pointA,endlineA]))
    return  ((pointA.distance(pointB),
             pointA.distance(intersection),
             pointB.distance(intersection)),
            intersection)
 
def plotResult(p1,p2,p3,nomeA,NomeB,distances):
     figure(num=None, figsize=(12, 9), dpi=80, facecolor='w', edgecolor='k')
     x_values = [p1[0], p3[0]]
     y_values = [p1[1], p3[1]]
     plt.plot(x_values, y_values,label='Visada '+nomeA)
     x_values = [p2[0], p3[0]]
     y_values = [p2[1], p3[1]]
     plt.plot(x_values, y_values,label='Visada '+nomeB)
     title = 'Intersection coordinates (X: {:10.2f}, Y: {:10.2f})\nTowers distance: {:10.2f}m\n'.format(
             p3[0],p3[1],distances[0])+\
             'Distance {0} to intersection: {1:10.2f}m\n'.format(nomeA,distances[1])+\
             'Distance {0} to intersection: {1:10.2f}m\n'.format(nomeB,distances[2])
     plt.title(title)
     plt.legend()
     plt.show()
 
def on_button_clicked(b):
  with output:
    distances, intersection = calcIntersection(xA,yA,azA,xB,yB,azB) 
    if not intersection.is_empty:
      plotResult([xA,yA],[xB,yB],[intersection.x,intersection.y],nomeA,nomeB,distances)
    else:
      print("There's no crossing point!")
  
button.on_click(on_button_clicked)
display(button, output)

Button(description='Calculate!', style=ButtonStyle())

Output()